# Insights de Negocio y Recomendaciones
## Predicción de Clientes Fidelizables en E-commerce

### Objetivos del Análisis de Negocio

Este notebook traduce los resultados técnicos del modelo en insights accionables para el negocio. Los objetivos específicos incluyen:

1. **Segmentación estratégica de clientes**
   - Clasificación basada en probabilidades de fidelización
   - Perfiles de comportamiento por segmento
   - Tamaño y valor de cada segmento

2. **Identificación de patrones de fidelización**
   - Factores clave que influyen en la lealtad
   - Características de clientes de alto valor
   - Señales tempranas de abandono

3. **Recomendaciones estratégicas**
   - Estrategias de retención por segmento
   - Optimización de campañas de marketing
   - Estimación de ROI potencial

4. **Preparación para Tableau**
   - Datasets optimizados para visualización
   - Campos calculados para dashboards
   - Documentación para stakeholders

### Marco de Análisis de Negocio

El análisis se estructura siguiendo principios de marketing analítico:
- **Segmentación basada en valor**: Priorización por impacto económico
- **Accionabilidad**: Recomendaciones implementables
- **Medibilidad**: KPIs claros para seguimiento
- **Escalabilidad**: Soluciones aplicables a gran escala

In [1]:
# Configuración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import joblib
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Importar utilidades para Tableau
import sys
sys.path.append('../src')
try:
    from utils.tableau_utils import prepare_tableau_export, export_tableau_documentation
    tableau_utils_available = True
except ImportError:
    tableau_utils_available = False
    print("Utilidades de Tableau no disponibles")

print("Librerías para análisis de negocio cargadas")
print(f"Fecha de análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Librerías para análisis de negocio cargadas
Fecha de análisis: 2025-09-01 13:44:14


In [2]:
# Cargar modelo y datos
try:
    best_model = joblib.load('../results/models/best_loyalty_model.pkl')
    scaler = joblib.load('../results/models/feature_scaler.pkl')
    selector = joblib.load('../results/models/feature_selector.pkl')
    
    with open('../results/models/model_info.json', 'r') as f:
        model_info = json.load(f)
    
    print(f"Modelo cargado: {model_info['model_name']}")
    
except FileNotFoundError:
    print("Modelos no encontrados. Creando configuración sintética...")
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import StandardScaler
    from sklearn.feature_selection import SelectKBest
    
    best_model = RandomForestClassifier(n_estimators=100, random_state=42)
    scaler = StandardScaler()
    selector = SelectKBest(k=10)
    
    model_info = {
        'model_name': 'Random Forest',
        'feature_columns': ['Recency', 'Frequency', 'Monetary', 'AvgRevenue']
    }

# Cargar dataset completo
try:
    df = pd.read_csv('../data/processed/customer_features_with_trends.csv')
    print(f"Dataset cargado: {df.shape[0]:,} clientes")
except FileNotFoundError:
    print("Dataset no encontrado. Creando datos sintéticos...")
    
    np.random.seed(42)
    n_customers = 1000
    
    df = pd.DataFrame({
        'CustomerID': range(12346, 12346 + n_customers),
        'Recency': np.random.exponential(30, n_customers),
        'Frequency': np.random.poisson(5, n_customers) + 1,
        'Monetary': np.random.gamma(2, 100, n_customers),
        'AvgRevenue': np.random.gamma(2, 50, n_customers),
        'UniqueProducts': np.random.poisson(8, n_customers) + 2,
        'Country': np.random.choice(['United Kingdom', 'France', 'Germany'], n_customers, p=[0.7, 0.15, 0.15])
    })
    
    # Variable objetivo
    df['IsLoyal'] = (
        (df['Frequency'] >= df['Frequency'].quantile(0.6)) & 
        (df['Monetary'] >= df['Monetary'].quantile(0.4)) & 
        (df['Recency'] <= df['Recency'].quantile(0.6))
    ).astype(int)
    
    print(f"Dataset sintético creado: {df.shape[0]:,} clientes")

print(f"Distribución actual: {df['IsLoyal'].value_counts().to_dict()}")

Modelo cargado: Random Forest
Dataset cargado: 4,338 clientes
Distribución actual: {0: 2494, 1: 1844}


## 1. Generación de Predicciones para Todos los Clientes

### Aplicación del Modelo a la Base Completa

Aplicamos el modelo entrenado a toda la base de clientes para obtener probabilidades de fidelización y generar segmentaciones estratégicas.

In [3]:
# Preparar datos para predicción
feature_columns = model_info.get('feature_columns', ['Recency', 'Frequency', 'Monetary', 'AvgRevenue'])
available_features = [col for col in feature_columns if col in df.columns]

X_full = df[available_features].fillna(0)

# Entrenar transformadores si es necesario
if not hasattr(scaler, 'scale_'):
    X_full_scaled = scaler.fit_transform(X_full)
    X_full_selected = selector.fit_transform(X_full_scaled, df['IsLoyal'])
    
    # Entrenar modelo si es necesario
    if not hasattr(best_model, 'feature_importances_'):
        best_model.fit(X_full_selected, df['IsLoyal'])
else:
    X_full_scaled = scaler.transform(X_full)
    X_full_selected = selector.transform(X_full_scaled)

# Generar predicciones
loyalty_predictions = best_model.predict(X_full_selected)
loyalty_probabilities = best_model.predict_proba(X_full_selected)[:, 1]

# Agregar predicciones al dataset
df['Loyalty_Prediction'] = loyalty_predictions
df['Loyalty_Probability'] = loyalty_probabilities

print("PREDICCIONES GENERADAS")
print("=" * 25)
print(f"Clientes procesados: {len(df):,}")
print(f"Predicciones positivas: {loyalty_predictions.sum():,} ({loyalty_predictions.mean():.1%})")
print(f"Probabilidad promedio: {loyalty_probabilities.mean():.3f}")
print(f"Rango de probabilidades: {loyalty_probabilities.min():.3f} - {loyalty_probabilities.max():.3f}")

# Distribución de probabilidades
fig = px.histogram(
    x=loyalty_probabilities,
    nbins=30,
    title='Distribución de Probabilidades de Fidelización',
    labels={'x': 'Probabilidad de Fidelización', 'y': 'Número de Clientes'}
)
fig.update_layout(height=400)
fig.show()

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- Country_encoded


## 2. Segmentación Estratégica de Clientes

### Definición de Segmentos de Valor

Creamos segmentos basados en probabilidad de fidelización y valor monetario para priorizar esfuerzos de marketing y retención.

In [ ]:
# Crear segmentos basados en probabilidad de fidelización
def create_loyalty_segments(prob):
    """
    Segmentación basada en probabilidad de fidelización
    """
    if prob >= 0.8:
        return 'Champions'
    elif prob >= 0.6:
        return 'Loyal Customers'
    elif prob >= 0.4:
        return 'Potential Loyalists'
    elif prob >= 0.2:
        return 'At Risk'
    else:
        return 'Lost Customers'

# Crear segmentos de valor monetario
df['Value_Segment'] = pd.qcut(
    df['Monetary'], 
    q=5, 
    labels=['Bronze', 'Silver', 'Gold', 'Platinum', 'Diamond']
)

# Aplicar segmentación de fidelización
df['Loyalty_Segment'] = df['Loyalty_Probability'].apply(create_loyalty_segments)

# Análisis de segmentos
segment_analysis = df.groupby('Loyalty_Segment').agg({
    'CustomerID': 'count',
    'Monetary': ['mean', 'sum'],
    'Frequency': 'mean',
    'Recency': 'mean',
    'UniqueProducts': 'mean',
    'Loyalty_Probability': 'mean'
}).round(2)

segment_analysis.columns = [
    'Num_Customers', 'Avg_Revenue', 'Total_Revenue', 
    'Avg_Frequency', 'Avg_Recency', 'Avg_Products', 'Avg_Probability'
]

# Calcular porcentajes
segment_analysis['Percentage'] = (segment_analysis['Num_Customers'] / len(df) * 100).round(1)
segment_analysis['Revenue_Share'] = (segment_analysis['Total_Revenue'] / segment_analysis['Total_Revenue'].sum() * 100).round(1)

print("ANÁLISIS DE SEGMENTACIÓN")
print("=" * 30)
display(segment_analysis)

# Visualización de segmentos
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribución de Clientes', 'Participación en Revenue', 
                   'Revenue Promedio por Segmento', 'Frecuencia vs Probabilidad'),
    specs=[[{"type": "pie"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# Distribución de clientes
fig.add_trace(
    go.Pie(labels=segment_analysis.index, values=segment_analysis['Num_Customers'],
           name="Clientes"),
    row=1, col=1
)

# Participación en revenue
fig.add_trace(
    go.Pie(labels=segment_analysis.index, values=segment_analysis['Total_Revenue'],
           name="Revenue"),
    row=1, col=2
)

# Revenue promedio
fig.add_trace(
    go.Bar(x=segment_analysis.index, y=segment_analysis['Avg_Revenue'],
           name="Avg Revenue"),
    row=2, col=1
)

# Scatter Frecuencia vs Probabilidad
fig.add_trace(
    go.Scatter(x=segment_analysis['Avg_Frequency'], y=segment_analysis['Avg_Probability'],
              mode='markers+text', text=segment_analysis.index,
              textposition="top center", name="Segmentos"),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Análisis Comprehensivo de Segmentos")
fig.show()